In [0]:
#Caminho datalake da camada raw/bronze
path = "raw/san/periodo/2025/10/16/extract_periodo_jdbc-san-maceio_20251016"
path = dbutils.widgets.get("data_lake_folder") 

In [0]:
%python
import json
#conexão oracle
jdbcSecret = "jdbc-san-maceio"
secret = dbutils.secrets.get("az-kv", jdbcSecret)
conn = json.loads(secret)
#string de conexão
driver = conn['driver']
jdbcUrl = conn['jdbcUrl']
user = conn['user']
pwd = conn['password']

#query principal fonte LIGACAORESUMO
query = """SELECT
CAST(P.ID AS NUMBER(38,0)) AS ID
,P.PERIODO AS PERIODO
,(SELECT UO.IF FROM FOZ_SAN.UNIDADEOPERACIONAL UO) AS IDUNIDADEOPERACIONAL
,SYSDATE
FROM FOZ_SAN.PERIODO P"""

df = (spark.read.format("jdbc")
  .option("driver", driver)
  .option("url", jdbcUrl)
  .option("user", user)
  .option("password", pwd)
  .option("query", query)
  .load())
  display(df.limit(10))


In [0]:
(df.write
  .format("parquet")
  .option("compression", "snappy")
  .mode("overwrite")
  .save(path))